# Imports.

In [ ]:
import tables
import numpy as np
import scipy
import holoviews as hv
hv.extension('bokeh')

## Get the template trace from a hdf5 file

In [ ]:
templateFile = '/reg/g/xpp/xpptut15/results/smalldata_tools/examples/notebooks/SingleAcqirisPeak_all.h5'
peakTemplate = tables.open_file(templateFile).root.sumTracePk
peakTemplate = tables.open_file(templateFile).root.singlePicked
peakTemplate = tables.open_file(templateFile).root.singlePicked_alt
peakTemplate = tables.open_file(templateFile).root.singlePicked_alt

peakMain = peakTemplate[50:200].copy()
peakMain = peakMain/peakMain.max()

## Peak Shape and position from sum of two templates.
Each peak is the sum of two templates, shifted by a pixel, the floating point part of the main peak position corresponds to the ratio of the two peaks. The figure below allows you to see the effect of the floating point position (which is also needed for the optimization to work as written).

In [ ]:
def templatePlot(peakpos, templateShape=150):
    args = [peakpos, min(peakpos-200,100), 1., 0.]
    template = templateArray(args, templateShape)
    c1 = hv.Curve(template,('time','time'),('V','V')).options(width=300)
    c2 = hv.Curve(template,('Time','Time'),('Volt','Volt')).redim.range(Time=(17,32),Volt=(0.3,1.)).options(width=250)
    return c1+c2

dmap = hv.DynamicMap(templatePlot, kdims=[hv.Dimension('peakpos',range=(19.75,21.25))])
dmap

## Functions for template fitting
The initial estimate step finds the largest derivitive as peak position and uses the maximum value after the peak as guess for the amplitude.

In [ ]:
def findPars(trace, nPeak=2):
    difftrace = np.diff(trace)
    peakIdx = scipy.signal.find_peaks_cwt(difftrace, widths=[3])
    maxIdx = [ x for _,x in sorted(zip(difftrace[peakIdx],peakIdx), key=lambda pair: pair[0], reverse=True)]
    peakPos=[];peakVal=[]
    for iPk in range(nPeak):
       idx=maxIdx[iPk]
       while difftrace[idx]>0:
            idx+=1
       peakPos.append(idx)
       peakVal.append(trace[idx])
    #sort in peak position.
    pPos = [ pos for pos,_ in sorted(zip(peakPos,peakVal))]
    pPk = [ pk for _,pk in sorted(zip(peakPos,peakVal))]
    for pVal in pPk:
        pPos.append(pVal)
    return pPos

#add template for peak w/ peak shifted by 1 with fraction of peak position.
#now the optimization actually works!
def templateArray(args, templateShape):
    template = peakMain#[10:110]
    templateMaxPos = np.argmax(template)
    if (args[0]>templateMaxPos):
        templatePk1 = np.append(np.zeros(args[0]-templateMaxPos), template)
    else:
        templatePk1 = template[templateMaxPos-args[0]:]
    if (templateShape-templatePk1.shape[0])>0:
        templatePk1 = np.append(templatePk1, np.zeros(templateShape-templatePk1.shape[0]))
    templatePkp1 = np.append(np.array([0]), templatePk1[:-1])
    frac1 = args[0]-int(args[0])
    template1 = templatePk1*(1.-frac1)+templatePkp1*frac1
    if args[3]==0:
        return template1*args[2]
    templatePk2 = np.append(np.zeros(args[1]-templateMaxPos), template)
    if (templateShape-templatePk2.shape[0])>0:
        templatePk2 = np.append(templatePk2, np.zeros(templateShape-templatePk2.shape[0]))
    templatePkp2 = np.append(np.array([0]), templatePk2[:-1])
    frac2 = args[1]-int(args[1])
    template2 = templatePk2*(1.-frac2)+templatePkp2*frac2
    templateSum=template1*args[2]+template2*args[3]
    return templateSum

#make sure to exclude points > max from calc!
def fitTemplateLeastsq(trace, saturationFrac=0.98, debug=False):
    args0 = findPars(trace)
    maxTrace = np.nanmax(trace)*saturationFrac
    if debug: print 'maxtrace: ',maxTrace,' n high pix ',(trace>maxTrace).sum() 
    if (trace>maxTrace).sum() > 2:
        maskSaturation=[trace<maxTrace]
    else:
        maskSaturation=np.ones_like(trace).astype(bool)
    errorfunction = lambda p: templateArray(p, trace.shape[0])[maskSaturation]-trace[maskSaturation]
    p, success = scipy.optimize.leastsq(errorfunction, args0)
    return p

def residuals(ar1, ar2, saturationFrac=0.98, error=9e-4):
    resid=0.
    maxTrace = np.nanmax(ar1)*saturationFrac
    if (ar1>maxTrace).sum() > 2:
        mask=np.array([ar1<maxTrace]).squeeze()
    else:
        mask=np.ones_like(ar1).astype(bool)
    ndof=0
    for r1,r2,m in zip(ar1, ar2, mask):
        #print 'r1, r2. resid :', r1, r2, m, resid, ndof
        diffSq = (r1-r2)/error*(r1-r2)/error*m
        if m != 0:
            resid+=diffSq
            ndof+=1

    return resid/(ndof-1)

## Load the traces to be fitted.

In [ ]:
expname='xcsx31116'
dirname='/reg/d/psdm/%s/%s/hdf5/smalldata/'%(expname[0:3],expname)

#run=101
#run=102
run=106
fname='sml_uxi_pedSub_%s_%04d_new.h5'%(expname, run)
Dat = tables.open_file('%s/%s'%(dirname, fname)).root
acqTraces = Dat.acq01.data.read().squeeze()

# Fit some example curves.

In [ ]:
trace1 = acqTraces[12]
trace2 = acqTraces[18]
xrange=(3100,3300)
#print findPars(trace1)
dimx=('xt','Time (trace1)')
dimy=('yt','Volt (trace1)')
dimx2=('xt2','Time (trace2)')
dimy2=('yt2','Volt (trace2)')

fitPars1 = fitTemplateLeastsq(trace1)
fitPars2 = fitTemplateLeastsq(trace2)
estCurve1 = templateArray(findPars(trace1), trace1.shape[0])
fitCurve1 = templateArray(fitPars1, trace1.shape[0])
resid1 = residuals(trace1[3125:3300], fitCurve1[3125:3300])
estCurve2 = templateArray(findPars(trace2), trace2.shape[0])
fitCurve2 = templateArray(fitPars2, trace2.shape[0])
resid2 = residuals(trace2[3125:3300], fitCurve2[3125:3300])

print resid1, resid2

hv.Curve(trace1,dimx,dimy,label='data').redim.range(xt=xrange).options(width=400) *\
hv.Curve(estCurve1,dimx,dimy,label='estimate').redim.range(xt=xrange) *\
hv.Curve(fitCurve1,dimx,dimy,label='fit').redim.range(xt=xrange) +\
hv.Curve(trace2,dimx2,dimy2).redim.range(xt2=xrange).options(width=400) *\
hv.Curve(estCurve2,dimx2,dimy2).redim.range(xt2=xrange) *\
hv.Curve(fitCurve2,dimx2,dimy2).redim.range(xt2=xrange)